# Notebook do Jeff.

Atividades da análise exploratória, de acordo com as notas de aula do Ogasawara:

## Identificar data objects (entidades e seus atributos)
Registrar o tipo de cada atributo (Nominal, binário, numérico)
*nominal: tipos de coisas
*binary: 0 ou 1, S e N (lembrando que pode ser simétrico ou assimétrico)
*ordinal: valores que representam relação de ordenação (ranking) das entidades ex: tamanho: p-m-g
*quantidade: numeros não discretos
*intervalo: medidas em uma escala sem um zero-point. ex> temperatura, datas do calendario, etc
*razão:  medidas em uma escala com um zero-point: ex: graus em celsius, quantidades monetárias, etc.

## Fazer descrições estatísticas dos dados:
Centralidade e características de dispersão (Ex. mediana, max, min, quantis, outliers, variancia, etc).
Dimensões numéricas: 
    dispersão em intervalos de multipla granularidade: boxplots em intervalos computados. 
    dispersão em métricas computadas: boxplots no cubo transformado (datawarehouse)
 
Exemplos de aula: 
*média, mediana, moda
*distribuição normal, assimétrica (positiva/negativa)
*quartis, faixa inter-quartis, sumário de 5 numeros (min, q1, mediana, q2, max): boxplot
*variância e desvio-padrão.

Fazer a descrição através de gráficos boxplot, histogramas, qualtile plot, scatter plot.

**Identificar outliers**: valores maiores que Q3 + 1,5*IQR ou menores que Q1 - 1,5*IQR

## Aplicar outras técnicas
Matriz de dados  
Matriz de dissimilaridade  
 (ver slides sobre Medidas de proximidade e dissimilaridade)





## Arquivos de dados  
Conforme descrição do Ogasawara em [Brazilian Flight Dataset](http://eic.cefet-rj.br/~eogasawara/brazilian-flight-datasets/):  
**airports-br.xlsx** - Lista de aeroportos. Não fornecido. Usaremos o do Gustavo: [airports.dat](https://openflights.org/data.html).  
**vra.RData** - todos os dados sobre voos, de jan/2009 a dez/2017.  
**airports_hist.xlsx** - Não fornecido. Seriam os histogramas das atividades dos 62 principis aeroportos, correspondendo a 92% do tráfego monitorado pela ANAC.  
**wu.RData** - informações sobre o clima nos principais 62 aeroportos.  
**vra-wu.RData** - informações sobre os voos, com dados do clima na partida e chegada nessses 62 aeroportos.  




### Arquivo **vra.RData**

In [8]:
load("dados/vra.RData") 
head(vra)

destiny,origin,airlines,flight,autho_code,line_type,depart_expect,depart,arrival_expect,arrival,⋯,arrival_delay,duration_expect,duration,duration_delta,name.x,city.x,state.x,name.y,city.y,state.y
CYYZ,SBGR,ACA,91,0,I,2009-01-11 22:30:00,2009-01-11 22:30:00,2009-01-12 08:35:00,2009-01-12 08:35:00,⋯,0,605,605,0,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,NA,NA,NA
CYYZ,SBGR,ACA,91,0,I,2009-01-05 22:30:00,2009-01-05 22:30:00,2009-01-06 08:35:00,2009-01-06 08:35:00,⋯,0,605,605,0,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,NA,NA,NA
CYYZ,SBGR,ACA,91,0,I,2009-01-26 22:30:00,2009-01-26 22:30:00,2009-01-27 08:35:00,2009-01-27 08:35:00,⋯,0,605,605,0,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,NA,NA,NA
CYYZ,SAEZ,ACA,95,0,I,2009-01-31 21:05:00,2009-01-31 21:05:00,2009-02-01 08:40:00,2009-02-01 08:40:00,⋯,0,695,695,0,NA,NA,NA,NA,NA,NA
CYYZ,SBGR,ACA,91,0,I,2009-01-31 22:30:00,2009-01-31 22:30:00,2009-02-01 08:35:00,2009-02-01 08:35:00,⋯,0,605,605,0,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,NA,NA,NA
CYYZ,SBGR,ACA,91,0,I,2009-01-14 22:30:00,2009-01-14 22:30:00,2009-01-15 08:35:00,2009-01-15 08:35:00,⋯,0,605,605,0,GUARULHOS - GOVERNADOR ANDRÉ FRANCO MONTORO,GUARULHOS,SP,NA,NA,NA


### Arquivo **wu.RData**

In [9]:
load("dados/wu.RData") 
head(wu)

data.airport,data.date,data.hour,data.temperature,data.dew_point,data.humidity,data.pressure,data.visibility,data.events,data.conditions
SBGR,2009-01-01,00:00,20,19,94,1016,NA,,Mist
SBGR,2009-01-02,00:00,20,19,94,1015,NA,,Clear
SBGR,2009-01-03,00:00,21,20,94,1012,10,Rain,Rain
SBGR,2009-01-04,00:00,21,19,94,1008,10,"Rain , Thunderstorm",Thunderstorms and Rain
SBGR,2009-01-05,00:00,16,16,100,1015,10,Rain,Mostly Cloudy
SBGR,2009-01-06,00:00,16,14,88,1017,10,,Mostly Cloudy


## Questões suscitadas
1. Qual é o percentual de voos atrasados em relação ao total 
  1. por ano?
  1. por aeroporto?
  1. por companhia aérea?
  1. por mês do ano?
  1. por umidade/temperatura/pressão?
  1. por visibilidade?
  1. por condições climáticas?
1. É possível identificar a propagação de atrasos (ex. atraso em POA = atraso em CGH)?
1. É possível identificar algum *trigger* meteorologico que sirva como preditor de atrasos?
1. É possível correlacionar esse trigger com os atrasos (Ex. CHUVA em POA = atraso em CGH)?
1. Quais foram as datas com maior quantidade de delays?
1. Há correlação entre as condições meteorológicas e os atrasos? (ou haverá outros motivos que os expliquem?)
1. Quais são as justificativas mais frequentes para os atrasos?


In [10]:
summary(wu)

 data.airport        data.date          data.hour         data.temperature
 Length:4438934     Length:4438934     Length:4438934     Min.   : 0.00   
 Class :character   Class :character   Class :character   1st Qu.:20.00   
 Mode  :character   Mode  :character   Mode  :character   Median :24.00   
                                                          Mean   :23.39   
                                                          3rd Qu.:27.00   
                                                          Max.   :90.00   
                                                          NA's   :56735   
 data.dew_point   data.humidity    data.pressure    data.visibility  
 Min.   : 0.00    Min.   :  0.00   Min.   :   0.0   Min.   :    0.0  
 1st Qu.:15.00    1st Qu.: 60.00   1st Qu.:1011.0   1st Qu.:   10.0  
 Median :19.00    Median : 74.00   Median :1014.0   Median :   10.0  
 Mean   :18.19    Mean   : 70.78   Mean   : 972.9   Mean   :   64.8  
 3rd Qu.:22.00    3rd Qu.: 88.00   3rd Qu.:1017.0 

WU: 
1. converter o campo *data.date* de character para date 
1. decompor o novo campo em ano, mes, dia
1. converter o campo *data.time* de character para time
1. decompor o novo campo em hora, minuto


In [11]:
summary(vra)

    destiny            origin           airlines           flight        
 SBGR   :1207826   SBGR   :1205293   GLO    :2876430   3893   :   14602  
 SBSP   : 786220   SBSP   : 783266   TAM    :2583614   3892   :   13973  
 SBBR   : 705285   SBBR   : 707281   AZU    :1720838   1939   :   13062  
 SBGL   : 613967   SBGL   : 612576   TIB    : 584460   1938   :   12948  
 SBKP   : 523274   SBKP   : 523276   ONE    : 534429   922    :   11912  
 SBCF   : 488655   SBCF   : 488099   PTB    : 247070   1501   :   11189  
 (Other):6192001   (Other):6197437   (Other):1970387   (Other):10439542  
   autho_code        line_type       depart_expect                
 0      :9990125   N      :6142110   Min.   :2009-01-01 00:00:00  
 2      : 225985   R      :1766731   1st Qu.:2011-06-14 14:44:30  
 4      : 132722   I      :1462271   Median :2013-06-30 17:25:00  
 9      :  46157   E      : 652614   Mean   :2013-07-19 15:40:17  
 6      :  43028   G      : 335633   3rd Qu.:2015-09-13 22:30:00  
 7    